In [ ]:
# 환경 세팅
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
drive.mount('/content/drive')

# 'MyDrive' 경로를 추가해야 해!
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/credit_preprocessing_total.parquet', engine='pyarrow')
# 잘 불러와졌는지 확인
print("데이터 로드 완료!")
df.info()

df_test = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/test_preprocessing_total.parquet', engine='pyarrow')
# 잘 불러와졌는지 확인
print("데이터 로드 완료!")
df_test.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
데이터 로드 완료!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Columns: 858 entries, 기준년월 to 혜택수혜율_B0M
dtypes: float32(57), float64(4), int16(148), int32(147), int8(452), object(50)
memory usage: 4.5+ GB
데이터 로드 완료!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Columns: 857 entries, 기준년월 to 혜택수혜율_B0M
dtypes: float32(57), float64(4), int16(153), int32(139), int8(455), object(49)
memory usage: 1.1+ GB


In [ ]:
pkgs_path = '/content/drive/MyDrive/colab_pip'

# 경로를 리스트의 가장 앞(0번 인덱스)에 추가해서 우선순위를 높임
if pkgs_path not in sys.path:
    sys.path.insert(0, pkgs_path)

try:
    import catboost
    print(f"성공! 버전: {catboost.__version__}")
except ImportError:
    print("여전히 못 찾고 있어. 폴더 구조를 다시 확인해봐야 할 것 같아.")


성공! 버전: 1.2.8


In [ ]:
# 경로 설정
save_path = '/content/drive/MyDrive/Colab Notebooks/'

# 불러오기 코드
loaded_mapping = {}
target_lists = ['Missing>50%', 'Constant', 'Imbalanced', 'FinalDrop']

for list_name in target_lists:
    file_name = f"{list_name}.txt"
    full_path = os.path.join(save_path, file_name)

    try:
        with open(full_path, 'r', encoding='utf-8') as f:
            loaded_mapping[list_name] = [line.strip() for line in f.readlines()]
        print(f"📂 드라이브에서 불러오기 성공: {file_name}")
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없어: {full_path}")

# 변수 재할당
over_50_missing_cols = loaded_mapping.get('Missing>50%', [])
constant_cols = loaded_mapping.get('Constant', [])
imbalanced_col_names = loaded_mapping.get('Imbalanced', [])
final_drop_list = loaded_mapping.get('FinalDrop', [])

# 1. 파일 경로 설정 (주신 코드를 그대로 활용했어!)
full_path = os.path.join(save_path, 'zero_importance_features 5000.txt')

# 2. 파일 읽어서 zero_importance_features 리스트에 할당하기
try:
    with open(full_path, 'r', encoding='utf-8') as f:
        # strip()을 사용해 양쪽 공백과 줄바꿈을 제거하고, 빈 줄은 제외하는 방식이야
        zero_importance_features = [line.strip() for line in f if line.strip()]

    print(f"성공적으로 {len(zero_importance_features)}개의 피처를 불러왔어! 😊")

except FileNotFoundError:
    print("어라, 파일 경로가 맞는지 확인해봐야 할 것 같아! 파일을 찾을 수 없대. 😅")

# set으로 변환해서 교집합 찾기
suspect = list(set(over_50_missing_cols) | set(imbalanced_col_names)) #여기를 변경

print(f"공통 항목 개수: {len(suspect)}개")
print(f"의심 항목 리스트: {suspect}")

📂 드라이브에서 불러오기 성공: Missing>50%.txt
📂 드라이브에서 불러오기 성공: Constant.txt
📂 드라이브에서 불러오기 성공: Imbalanced.txt
📂 드라이브에서 불러오기 성공: FinalDrop.txt
성공적으로 165개의 피처를 불러왔어! 😊
공통 항목 개수: 369개
의심 항목 리스트: ['컨택건수_카드론_인터넷_B0M', '평잔_RV일시불_해외_3M', '컨택건수_카드론_청구서_R6M', '인입불만월수_IB_R6M', '이용건수_당사기타_B0M', 'RP건수_건강_B0M', '잔액_현금서비스_B0M', '이용금액_R3M_체크_가족', '기본연회비_B0M', '할부금액_유이자_14M_R12M', '인입횟수_IB_B0M', '최대이용금액_카드론_R12M', '증감_RP건수_렌탈_전월', '_2순위여유업종', '할부금액_무이자_12M_R12M', '소지여부_신용', '이용건수_B페이_B0M', '이용횟수_선결제_B0M', '포인트_포인트_월적립_B0M', '자발한도감액후경과월', '잔액_할부_해외_B0M', '컨택건수_카드론_당사앱_R6M', '우수회원면제카드수_B0M', 'IB문의건수_CS_R6M', '할부금액_부분_3M_R12M', 'IB문의건수_APP_B0M', '시장연체상환여부_R6M', '_3순위여유업종_이용금액', '_2순위교통업종', 'IB상담건수_VOC불만_B0M', '방문일수_모바일웹_B0M', '당사PAY_방문월수_R6M', '컨택건수_카드론_EM_R6M', '연체잔액_할부_B0M', '평잔_할부_해외_6M', 'IB문의건수_카드발급_B0M', '제휴연회비_B0M', '컨택건수_이용유도_TM_B0M', '이용금액_카드론_B0M', '할부금액_무이자_14M_R12M', '이용금액_D페이_R3M', '컨택건수_리볼빙_LMS_B0M', '이용건수_D페이_R3M', 'IB문의건수_정보변경_R6M', '여유_Pet이용금액', '컨택건수_부대서비스_TM_B0M', '연체일수_최근', '잔액_리볼빙CA이월_B0M', '컨택건

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

!pip install -U imbalanced-learn

# --- [전처리 추가] SMOTE 적용을 위한 준비 ---
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split

# 0: Major (C, 2, 3), 1: Minor (4, 5)
major_classes = [1, 2, 3]
minor_classes = [4, 5]

# 1. 타겟 매핑 및 컬럼 제거
target_map = {'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1}
df['Segment'] = df['Segment'].map(target_map)
df = df.dropna(subset=['Segment'])

drop_cols = ['ID', '기준년월'] + suspect #여기를 변경
X = df.drop(columns=drop_cols + ['Segment'])
y = df['Segment']
X_test = df_test.drop(columns=drop_cols)

# --- [수정 포인트 1] fillna 제거 및 타입 강제 변환 ---
# 800개 컬럼에 fillna를 하는 대신, CatBoost가 인식할 수 있게 문자열 타입으로만 바꿉니다.
# 'None'으로 채우는 것보다 astype(str)이 더 효율적이며, 결측치는 자동으로 ㅁ'nan' 문자열이 됩니다.
cat_features = X.columns[X.dtypes == 'object'].tolist()
for col in cat_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

# 2. Train-Validation Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [ ]:


# 1. 범주형 변수의 '인덱스(위치)' 찾기
# SMOTENC는 컬럼 이름이 아니라 '몇 번째 컬럼이 범주형인지' 숫자로 알려줘야 해.
# cat_features는 리스트 형태여야 함 (예: ['성별', '직업', ...])
cat_indices = [X_train.columns.get_loc(col) for col in cat_features]

# 2. 결측치(NaN) 처리
# SMOTE는 결측치가 있으면 작동하지 않아! CatBoost는 알아서 처리하지만 SMOTE를 쓰려면 채워야 해.
# 간단하게 문자열은 'Missing', 숫자는 -999 등으로 채우는 임시 조치를 취할게.
X_train_filled = X_train.copy()
X_train_filled[cat_features] = X_train_filled[cat_features].fillna('Missing')
X_train_filled = X_train_filled.fillna(-999) # 수치형 결측치 처리

# 3. SMOTE-NC 적용 (Minor 클래스 증강)
# sampling_strategy='auto'로 두면 가장 많은 클래스(C,D,E) 수만큼 A,B를 늘려줘.
# 너무 많다 싶으면 딕셔너리로 조절 가능해 (예: {5: 10000, 4: 10000})
smote_nc = SMOTENC(categorical_features=cat_indices, random_state=42, sampling_strategy={5: 10000, 4: 10000})

print("⏳ SMOTE-NC 적용 중... (시간이 좀 걸릴 수 있어!)")
X_train_resampled, y_train_resampled = smote_nc.fit_resample(X_train_filled, y_train)

print(f"SMOTE 적용 후 Train 데이터 크기: {X_train_resampled.shape}, {y_train_resampled.shape}")
print(f"적용 후 클래스 비율:\n{y_train_resampled.value_counts(normalize=True)}")

# 4. 데이터 원상복구 (선택사항)
# SMOTE 과정에서 범주형 데이터가 숫자로 바뀌거나 할 수 있어서,
# CatBoost에 넣기 전에 다시 타입을 맞춰주는 게 좋아 (특히 범주형).
# 하지만 SMOTENC는 보통 원래 형태를 유지하려고 노력하니까 일단 진행해보고 에러 나면 타입 변환이 필요해.

⏳ SMOTE-NC 적용 중... (시간이 좀 걸릴 수 있어!)


In [ ]:
# --- [Model 1] Gatekeeper: Binary Classification ---

# 1. 이진 타겟 생성 (Minor 그룹이면 1, 아니면 0)
y_train_bin = y_train_resampled.isin(minor_classes).astype(int)
y_val_bin = y_val.isin(minor_classes).astype(int)

# 2. 이진 분류용 가중치 계산 (0과 1의 비율에 맞춰서)
classes_bin = np.unique(y_train_bin)

weights_bin = compute_class_weight(class_weight='balanced', classes=classes_bin, y=y_train_bin)

class_weights_bin = dict(zip(classes_bin, weights_bin))

# 2. Pool 생성 (Resampled 데이터 사용)
train_pool_bin = Pool(data=X_train_resampled, label=y_train_bin, cat_features=cat_features)
val_pool_bin = Pool(data=X_val, label=y_val_bin, cat_features=cat_features)

# 4. 모델 학습 (이진 분류)
model_gatekeeper = CatBoostClassifier(
    iterations=5000, # 복잡한 문제가 아니므로 횟수는 좀 줄여도 됨
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss', # 이진 분류니까 Logloss
    eval_metric='AUC',       # 불균형 데이터라 AUC가 지표로 좋음
    class_weights=class_weights_bin,
    task_type='GPU',
    devices='0',
    random_seed=42,
    verbose=500,
    bootstrap_type='Poisson',
    max_ctr_complexity=1,
    gpu_ram_part=0.8,
    border_count=32
)

print("=== Gatekeeper 모델 학습 시작 ===")
model_gatekeeper.fit(train_pool_bin, eval_set=val_pool_bin, early_stopping_rounds=100)

In [ ]:
# --- [Model 2] Major Specialist: 1, 2, 3 Classification ---

# 1. Major 그룹 데이터만 필터링 (불리언 인덱싱 사용)
mask_train_major = y_train.isin(major_classes)
mask_val_major = y_val.isin(major_classes)

X_train_major = X_train[mask_train_major]
y_train_major = y_train[mask_train_major]
X_val_major = X_val[mask_val_major]
y_val_major = y_val[mask_val_major]

# 2. 가중치 재계산 (이제 1, 2, 3끼리만 경쟁하므로 여기서 다시 밸런싱)
classes_major = np.unique(y_train_major)
weights_major = compute_class_weight(class_weight='balanced', classes=classes_major, y=y_train_major)
class_weights_major = dict(zip(classes_major, weights_major))

# 3. Pool 생성
train_pool_major = Pool(data=X_train_major, label=y_train_major, cat_features=cat_features)
val_pool_major = Pool(data=X_val_major, label=y_val_major, cat_features=cat_features)

# 4. 모델 학습
model_major = CatBoostClassifier(
    iterations=25000,
    learning_rate=0.02, # 1,2,3 구분은 어려우니 천천히
    depth=6, # 복잡하면 depth를 8 정도로 늘려보는 것도 추천
    l2_leaf_reg=10,
    subsample=0.7,
    loss_function='MultiClass',
    eval_metric='Accuracy', # 혹은 MultiClass
    class_weights=class_weights_major,
    task_type='GPU',
    devices='0',
    random_seed=42,
    verbose=1000,
    cat_features=cat_features, # 명시적으로 넣어주는 게 안전

    bootstrap_type='Poisson',
    max_ctr_complexity=1,
    gpu_ram_part=0.8,
    border_count=32
)

print("=== Major 모델 학습 시작 ===")
model_major.fit(train_pool_major, eval_set=val_pool_major, early_stopping_rounds=500)

In [ ]:
# --- [Model 3] Minor Specialist: 4, 5 Classification ---

# 1. Minor 그룹 데이터만 필터링
mask_train_minor = y_train_resampled.isin(minor_classes)
mask_val_minor = y_val.isin(minor_classes)

X_train_minor = X_train_resampled[mask_train_minor]
y_train_minor = y_train_resampled[mask_train_minor]

X_val_minor = X_val[mask_val_minor]
y_val_minor = y_val[mask_val_minor]
# 2. 가중치 재계산 (4, 5번 간의 균형)
classes_minor = np.unique(y_train_minor)
weights_minor = compute_class_weight(class_weight='balanced', classes=classes_minor, y=y_train_minor)
class_weights_minor = dict(zip(classes_minor, weights_minor))

# Pool 생성
train_pool_minor = Pool(data=X_train_minor, label=y_train_minor, cat_features=cat_features)
val_pool_minor = Pool(data=X_val_minor, label=y_val_minor, cat_features=cat_features)

# 4. 모델 학습
model_minor = CatBoostClassifier(
    iterations=5000,
    learning_rate=0.001,    # 더 천천히, 정교하게 학습
    depth=2,               # 얕은 깊이는 유지 (데이터가 적으므로)
    l2_leaf_reg=15,         # 정규화 강화
    loss_function='MultiClass',
    eval_metric='AUC',     # 더 민감한 지표로 변경
    class_weights=class_weights_minor,
    task_type='GPU',
    random_seed=42,
    verbose=500,
    early_stopping_rounds=200, # 성능 개선 없으면 200번 뒤에 멈춤
    cat_features=cat_features
)

print("=== Minor 모델 학습 시작 ===")
model_minor.fit(train_pool_minor, eval_set=val_pool_minor, early_stopping_rounds=100)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

print("=== 최종 검증(Validation) 진행 ===")

# === [수정된 최종 검증 로직] 확률 기반 소프트 보팅 ===

# 1. 모든 검증 데이터에 대해 각 모델의 '확률(Probability)'을 계산합니다.
# 주의: Major/Minor 모델에 '자기가 배우지 않은 데이터'가 들어가도,
# Gatekeeper 확률이 0에 가까우면 어차피 상쇄되므로 괜찮습니다.

# (N, 2) 형태: [Major확률, Minor확률]
probs_gate = model_gatekeeper.predict_proba(X_val)
prob_is_major = probs_gate[:, 0].reshape(-1, 1) # (N, 1)
prob_is_minor = probs_gate[:, 1].reshape(-1, 1) # (N, 1)

# (N, 3) 형태: [1번확률, 2번확률, 3번확률]
probs_major_specialist = model_major.predict_proba(X_val)

# (N, 2) 형태: [4번확률, 5번확률]
probs_minor_specialist = model_minor.predict_proba(X_val)

# 2. 계층적 확률 결합 (Chain Rule)
# 최종 확률 = Gatekeeper가 해당 그룹일 확률 * 전문 모델이 해당 클래스일 확률

# 1, 2, 3번 클래스의 최종 확률
final_probs_major = prob_is_major * probs_major_specialist

# 4, 5번 클래스의 최종 확률
final_probs_minor = prob_is_minor * probs_minor_specialist

# 3. 전체 확률 합치기 (N, 5)
# 순서대로 1, 2, 3, 4, 5번 컬럼으로 병합
final_probs_all = np.hstack([final_probs_major, final_probs_minor])

# 4. 가장 높은 확률을 가진 클래스 선택 (Argmax)
# 인덱스 0->1번, 1->2번... 이므로 +1을 해줍니다. (Target Map에 따라 조정 필요)
# 작성하신 코드의 target_map이 {'A': 5, ... 'E': 1} 이므로,
# model_major.classes_ 와 model_minor.classes_ 순서를 확인해야 합니다.
# 보통 정렬되어 나오므로 1,2,3 / 4,5 순서일 것입니다.

final_pred_indices = np.argmax(final_probs_all, axis=1)

# 클래스 매핑 복원 (0,1,2 -> 1,2,3 / 3,4 -> 4,5)
# Major(1,2,3) + Minor(4,5) 순서로 hstack 했으므로:
# 인덱스 0 -> Class 1
# 인덱스 1 -> Class 2
# 인덱스 2 -> Class 3
# 인덱스 3 -> Class 4
# 인덱스 4 -> Class 5
final_predictions = final_pred_indices + 1

# 5. 결과 출력
print("\n=== Soft Voting 계층적 분류 최종 성적표 ===")
print(classification_report(y_val, final_predictions))

cm = confusion_matrix(y_val, final_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[1,2,3,4,5])
disp.plot(cmap='Blues')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from catboost import Pool

print("=== 테스트 데이터 예측 시작 (Pool 사용) ===")

# 1. 테스트 데이터도 Pool 객체로 만들기
# 학습 때 사용한 cat_features 정보를 그대로 넣어줘야 해!
test_pool = Pool(data=X_test, cat_features=cat_features)

# 2. Gatekeeper가 먼저 판단 (Minor일 확률 계산)
# predict_proba에 X_test 대신 test_pool을 넣습니다.
gatekeeper_test_probs = model_gatekeeper.predict_proba(test_pool)[:, 1]

# 3. 임계값(Threshold) 적용 (Validation 때 설정한 0.8 유지)
threshold = 0.80
is_minor_test = (gatekeeper_test_probs > threshold)
is_major_test = (gatekeeper_test_probs <= threshold)

print(f"Gatekeeper 분류 결과: Major행 {sum(is_major_test)}개 / Minor행 {sum(is_minor_test)}개")

# 4. 결과를 담을 빈 배열 생성 (초기화)
# 나중에 채워넣을 공간을 미리 만들어두는 거야.
final_test_predictions = np.zeros(len(X_test))

# 5. Major 그룹 예측
if np.sum(is_major_test) > 0:
    # Major로 분류된 데이터만 추려내기
    X_test_major = X_test[is_major_test]

    # 추려낸 데이터로 새로운 Pool 만들기 (중요!)
    test_pool_major = Pool(data=X_test_major, cat_features=cat_features)

    # 예측 수행
    preds_major = model_major.predict(test_pool_major)
    final_test_predictions[is_major_test] = preds_major.flatten()

# 6. Minor 그룹 예측
if np.sum(is_minor_test) > 0:
    # Minor로 분류된 데이터만 추려내기
    X_test_minor = X_test[is_minor_test]

    # 추려낸 데이터로 새로운 Pool 만들기
    test_pool_minor = Pool(data=X_test_minor, cat_features=cat_features)

    # 예측 수행
    preds_minor = model_minor.predict(test_pool_minor)
    final_test_predictions[is_minor_test] = preds_minor.flatten()

# 7. 숫자 -> 알파벳 역변환 (Inverse Mapping)
# 학습할 때: {'A': 5, 'B': 4, 'C': 3, 'D': 2, 'E': 1}
inv_target_map = {5: 'A', 4: 'B', 3: 'C', 2: 'D', 1: 'E'}

# 8. 제출용 데이터프레임 생성
submission = pd.DataFrame({
    'ID': df_test['ID'],
    'Segment': final_test_predictions
})

# 숫자를 알파벳으로 매핑
submission['Segment'] = submission['Segment'].map(inv_target_map)

# 9. 결측치 확인
if submission['Segment'].isnull().sum() > 0:
    print(f"경고: {submission['Segment'].isnull().sum()}개의 값이 변환되지 않았어! 확인이 필요해.")
else:
    print("모든 데이터가 성공적으로 변환되었어.")

# 1. ID별로 그룹화하여 Segment 컬럼에 최빈값(mode) 적용
# .mode()는 결과가 여러 개일 수 있어서 [0]으로 가장 첫 번째 값을 선택해.
submission_final = submission.groupby('ID')['Segment'].agg(lambda x: x.mode()[0]).reset_index()

# 2. 결과 확인
print(f"최종 제출 데이터 크기: {submission_final.shape}") # (100000, 2)가 나와야 해
print(submission_final['ID'].value_counts().max()) # 이제 1이 나와야 함

# 3. CSV로 저장
save_path = '/content/drive/MyDrive/Colab Notebooks/submission.csv'
submission_final.to_csv(save_path, index=False)

print(f"다수결 투표 완료 및 저장 성공: {save_path}")